# **Exponentially Weighted Moving Average**

In [67]:
import pandas as pd 
import tensorflow
import keras
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, accuracy_score

In [68]:
df = pd.read_csv('ad_click_dataset.csv')
df.head()

,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,670,User670,22.0,NaN,Desktop,Top,Shopping,Afternoon,1
1,3044,User3044,NaN,Male,Desktop,Top,NaN,NaN,1
2,5912,User5912,41.0,Non-Binary,NaN,Side,Education,Night,1
3,5418,User5418,34.0,Male,NaN,NaN,Entertainment,Evening,1
4,9452,User9452,39.0,Non-Binary,NaN,NaN,Social Media,Morning,0


In [69]:
df.shape

(10000, 9)

In [70]:
df.isnull().sum()

id                     0
full_name              0
age                 4766
gender              4693
device_type         2000
ad_position         2000
browsing_history    4782
time_of_day         2000
click                  0
dtype: int64

In [71]:
df.duplicated().sum()

2853

In [72]:
df.drop_duplicates(inplace=True)

In [73]:
df.shape

(7147, 9)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['id', 'full_name', 'click']),
                                                    df['click'],
                                                    test_size=0.2,
                                                    random_state=42)

In [75]:
numerical_columns = [0]
categorical_columns = [1, 2, 3, 4, 5]

In [76]:
def apply_ewma(X, span=3):
    X_df = pd.DataFrame(X)
    return X_df.ewm(span=span, adjust=False).mean().values 

In [77]:
handle_numerical = Pipeline(steps=[
    ('impute', KNNImputer(n_neighbors=11)),
    ('ewma', FunctionTransformer(apply_ewma)),
    ('scale', MinMaxScaler())
])

In [78]:
handle_categorical = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(handle_unknown='ignore'))
])

In [79]:
preprocessing = ColumnTransformer(transformers=[
    ('numerical', handle_numerical, numerical_columns),
    ('categorical', handle_categorical, categorical_columns)
])

In [80]:
X_train = preprocessing.fit_transform(X_train)
X_test = preprocessing.transform(X_test)

In [81]:
model = Sequential()
model.add(Dense(32, activation='relu'))

for i in range(1, 21):
    model.add(Dense(int(32 / i), activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [82]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [83]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.5126 - loss: 0.6931
Epoch 2/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5181 - loss: 0.6929
Epoch 3/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5045 - loss: 0.6931
Epoch 4/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5125 - loss: 0.6929
Epoch 5/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5075 - loss: 0.6931
Epoch 6/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5097 - loss: 0.6930
Epoch 7/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5227 - loss: 0.6924
Epoch 8/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5141 - loss: 0.6928
Epoch 9/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5184 - loss: 0.6926
Epoch 10/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5055 - loss: 0.6932
Epoch 11/100
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5063 - loss: 0.6931
Epoch 12/100
179/179 ━━━━━━━━━━━━━━━━━━━

In [84]:
y_pred = model.predict(X_test)

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


In [86]:
y_pred = (y_pred > 0.5).astype(int)

In [87]:
accuracy_score(y_test, y_pred)

0.5076923076923077

In [88]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       704
           1       0.51      1.00      0.67       726

    accuracy                           0.51      1430
   macro avg       0.25      0.50      0.34      1430
weighted avg       0.26      0.51      0.34      1430



c:\Users\alnafay\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alnafay\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alnafay\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera